In [7]:
import json
from kafka import KafkaProducer

# Kafka Producer 설정
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    batch_size=100,
    linger_ms=10,
    acks=1
)

In [86]:
import pytz
import random
import uuid
import pandas as pd
import datetime

sent_events = []

# 1. 이벤트 시간의 시작 지점 설정 (한국 시간 기준으로 설정)
# 예시: 현재 시각으로부터 2일 전, 오전 9시 정각
korea_tz = pytz.timezone('Asia/Seoul')
start_time_kst = korea_tz.localize(datetime.datetime(2025, 6, 30, 13, 26, 0, 0)) # 연, 월, 일, 시, 분, 초, 마이크로초

# 2. 각 이벤트마다 증가시킬 시간 간격 설정
# 예시: 각 이벤트마다 10초씩 증가 (테스트를 위해 짧게 설정)
time_increment = datetime.timedelta(seconds=30)

user_ids = ["111", "222", "333"]
num_events_per_user = 4 # 각 사용자당 보낼 이벤트 개수

current_event_time = start_time_kst

print(f"Starting event time generation from: {current_event_time.isoformat()}")

for _ in range(num_events_per_user * len(user_ids)): # 총 이벤트 개수만큼 반복
    # 사용자 ID를 순환하며 할당
    current_user_id = random.choice(user_ids) # user_ids[i % len(user_ids)] 등으로 순차 할당도 가능

    event = {
        "event_time": current_event_time.isoformat(), # ISO 8601 포맷으로 변환 (예: 2025-06-28T09:00:00+09:00)
        "event_type": random.choice(["view", "cart", "purchase"]),
        "product_id": str(random.randint(10000000, 99999999)),
        "category_id": str(random.randint(1000000000000000000, 9999999999999999999)),
        "category_code": random.choice(["electronics", "beauty", None]),
        "brand": random.choice(["apple", "samsung", "nike"]),
        "price": round(random.uniform(10, 500), 2),
        "user_id": current_user_id,
        "user_session": str(uuid.uuid4())
    }

    # 카프카로 이벤트 전송 (실제 사용 시 주석 해제)
    producer.send("ecommerce-events", value=event)
    sent_events.append(event)

    # 다음 이벤트를 위해 시간 증가
    current_event_time += time_increment

producer.flush() # 모든 메시지가 전송될 때까지 대기
sent_df = pd.DataFrame(sent_events)

Starting event time generation from: 2025-06-30T13:26:00+09:00


In [87]:
# DataFrame 전체 표시 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 전송된 데이터 전체 표시
display(sent_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2025-06-30T13:26:00+09:00,purchase,19843718,9858952802413925627,electronics,apple,281.88,333,9dc81063-86ac-47b8-b009-a35af660e78c
1,2025-06-30T13:26:30+09:00,purchase,72694566,5790837533602362078,None,samsung,357.79,222,293a666f-95ff-4fc1-a4a8-c2b2170a153c
2,2025-06-30T13:27:00+09:00,cart,86219393,9300852797422151858,electronics,apple,284.65,111,853dbbcb-82eb-4790-9b6f-bd5a4937db5d
3,2025-06-30T13:27:30+09:00,cart,35786397,4576451202244319835,beauty,samsung,342.83,222,fe4b9672-a5a4-49f8-8e0a-00f06c5c5049
4,2025-06-30T13:28:00+09:00,cart,90034427,2654215311512246202,beauty,apple,299.30,222,8469f7ad-cdea-457f-af30-5dcb13bdec41
5,2025-06-30T13:28:30+09:00,purchase,14073742,5467507705403636477,None,apple,95.29,333,20110288-efb3-4ac3-b6f6-b4a671de5bd7
6,2025-06-30T13:29:00+09:00,purchase,87248109,6437792966403236604,beauty,apple,51.82,333,3d36bb91-8f02-4036-aba8-25a35723e4ea
7,2025-06-30T13:29:30+09:00,view,51740256,5959188069533783042,None,samsung,143.55,333,6f2d856e-7663-434f-92aa-f211fc0d9fe3
8,2025-06-30T13:30:00+09:00,view,40647319,5994357190677859480,electronics,samsung,83.09,111,95fda5a2-fdd3-4e0f-86a9-a3e8d75dfb99
9,2025-06-30T13:30:30+09:00,purchase,43460452,1428864496744780209,electronics,nike,421.67,222,f5a12461-94ac-45ee-b1e8-7e304eb911f3


In [88]:
# Redis 연결 및 키 확인
import redis
import json
from pprint import pprint

# Redis 클라이언트 생성
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

try:
    r.ping()
    print('Redis 연결 성공')
except Exception as e:
    print(f"Redis 연결 실패: {e}")


Redis 연결 성공


In [89]:
# 1. 모든 user_features 키 조회
user_keys = r.keys("*")
for key in user_keys[:10]:  # 처음 10개만 표시
    print(f"- {key}")


- user_features:222
- user_features:111
- user_features:333


In [90]:
r.hgetall("user_features:222")

{'time_slot_0_6': '0.0',
 'user_id': '222',
 'time_slot_18_24': '0.0',
 'time_slot_6_12': '0.0',
 'time_slot_12_18': '1.0',
 'timestamp': '1751466402972'}